# Tow-Based Discontinuous Composite (TBDC) property predictor
This code is written for an MSc research project at Imperial College London

## Mount drive

In [35]:
# Data to be imported from google drive
from google.colab import drive
drive.mount('/content/drive')

# Set path for drive with datasets
drive_path ="/content/drive/MyDrive/Individual_project_ICL"

Mounted at /content/drive


## Imports and installs

In [ ]:
import sys
!{sys.executable} -m pip install tensorflow
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install pandas

In [6]:
import os
import random
import time
import math
import datetime
import shutil
import json

import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


## Import data


In [36]:
# Read data from json file in google drive
dataPath = os.path.join(drive_path,'dataDumpTest.json')

with open(dataPath) as json_file: # load into dict
    dat = json.load(json_file)

params = list(dat.keys()) # get list of parameters in file
print(params)



# # Read data from txt file in format "parameter:value"\n e.g. N_t:[8, 8]\n
# with open(drive_path + '/datDumpTest.json') as f:
#     dat = f.read()
# dat = dat.splitlines() # split into string array by linebreaks

# namesToLoad = ['theta_t', 'failure_factors'] # set variables to load
# datDict = {} # create dictionary to store data, for easier debugging

# for n in namesToLoad: # For each variable to be loaded
#     paramStr = [match for match in dat if n+':' in match] # find all string array entries with the given parameter name (use ":" to ensure only the actual variable is taken)
#     varname = paramStr[0].split(':')[0] # Set variable name for dictionary
#     datDict[varname] = paramStr[0].split(':')[-1] # assign key and value for parameter in dictionary
# # np.array()
# theta_t = datDict['theta_t'] # Angle of each ply in laminate
# failure_factors = datDict['failure_factors'] #

# failure_factors = np.array(list(failure_factors), dtype=float)
# print(failure_factors)

['theta_t', 'Q_g', 'overlap_lengths', 'overlap_thicknesses', 'sigma_l', 'failure_factors']


In [37]:
theta_t = np.array(dat['theta_t'])
Q_g = np.array(dat['Q_g'])
overlap_lengths = np.array(dat['overlap_lengths'])
overlap_thicknesses = np.array(dat['overlap_thicknesses'])
sigma_l = np.array(dat['sigma_l'])
failure_factors = np.array(dat['failure_factors'])

In [38]:
theta_t

array([[ 0.11242025,  1.38040184,  0.75478414, -0.49195673,  0.94975915,
        -1.55676126, -0.27682871, -0.27787572],
       [-0.80557676, -0.2776038 , -0.10155421,  0.3015842 ,  1.34131826,
        -1.54408707,  0.58415655,  0.45935336],
       [-0.26237673,  0.10008502, -1.32332291, -0.01149989, -1.02529069,
         1.41184624,  0.88175597,  0.38931997],
       [-1.14274662,  0.41468062,  0.76140174,  1.01868181,  0.60049487,
        -1.55242157, -1.38890227,  0.00460712],
       [ 1.43882036, -0.6394743 , -1.36952088, -0.53373687, -0.88292747,
        -1.31944733, -0.95899611, -1.32360155],
       [-0.44571237, -0.34913086, -0.46934769, -0.71424056, -0.5286414 ,
         0.60033129,  1.45541765, -0.15200357],
       [ 1.22033729, -0.82586065,  0.66475885, -1.31163401,  0.45431579,
         1.13882897, -1.27079639, -0.49311739],
       [ 0.17182728,  1.38758495, -0.46217544,  0.661181  , -1.20044883,
        -0.06609314,  0.18638871,  0.98601462],
       [-0.53766235,  1.44415708